In [1]:
import datasets 
import torch 
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset
from tokenizers.normalizers import NFKC
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer, trainers, models
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaForMaskedLM, RobertaConfig
from transformers import PreTrainedTokenizerFast, RobertaTokenizerFast

import os 
from typing import List
from collections import defaultdict

os.environ['CUDA_VISIBLE_DEVICES'] = "6,7"

DATA_PATH = "/Volumes/New Volume/malware-detection-dataset/opcodes/processed-data"
MAX_LENGTH = 64

In [ ]:
def get_data(path: os.PathLike, full_path: bool = True) -> List[str]:
    all_files = os.listdir(path)
    
    if full_path:
        return [os.path.join(path, file) for file in all_files if file.endswith('.txt') and not file.startswith("._")]
    else: 
        return all_files

def get_labels(filenames):
    return [1 if "VirusShare" in filename else 0 for filename in filenames]

# paths = get_data(DATA_PATH)
# labels = get_labels(paths)

In [ ]:
class OpcodeDataset(Dataset): 
    def __init__(self, paths, labels):
        assert len(paths) == len(labels), "Mismatch between number of files and labels"
        self.paths = paths 
        self.labels = labels

    def __len__(self):
        return len(self.paths)        


    def __getitem__(self, idx):
        assert 0 <= idx <= len(self), "Index out of range"
        label = self.labels[idx]

        with open(self.paths[idx], 'r') as file: 
            content = file.readlines() 
            
        return ' '.join([opcode.rstrip() for opcode in content]), label

# opcode_dataset = OpcodeDataset(paths, labels)

In [2]:
if not os.path.exists('./MalBERTa'):
    tokenizer = Tokenizer(models.WordLevel(unk_token="<unk>"))
    tokenizer.normalizer = NFKC()
    tokenizer.pre_tokenizer = Whitespace()

    trainer = trainers.WordLevelTrainer(
        vocab_size=1293, 
        special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>",
        ], 
    )
    tokenizer.train(paths, trainer)
    tokenizer.save('MalBERTa/tokenizer.json')

    hf_tokenizer = PreTrainedTokenizerFast(
        tokenizer_file="MalBERTa/tokenizer.json",
        unk_token="<unk>",
        bos_token="<s>",
        eos_token="</s>",
        pad_token="<pad>",
        mask_token="<mask>"
    )
    hf_tokenizer.save_pretrained("MalBERTa")
    tokenizer = hf_tokenizer
else: 
    tokenizer = PreTrainedTokenizerFast.from_pretrained("MalBERTa")

In [3]:
def dataset_generator():
    for text, label in tqdm(opcode_dataset): 
        yield {
            "text": text,
            "label": label
        }

if not os.path.exists('./data/raw'):
    dataset = datasets.Dataset.from_generator(dataset_generator)
    dataset = dataset.train_test_split(test_size=0.2)
    dataset.save_to_disk("data/raw")
else: 
    dataset = datasets.load_from_disk("./data/raw")

Saving the dataset (2/2 shards): 100%|██████████| 1388/1388 [00:00<00:00, 2638.87 examples/s]


In [4]:
def handle_sample(sample):
    texts = sample['text']
    labels = sample['label']
    
    flattened = defaultdict(list)

    for text, label in zip(texts, labels):
        tokenized = tokenizer(
            text,
            padding='max_length',
            max_length=MAX_LENGTH,
            return_overflowing_tokens=True,
            truncation=True,
            return_special_tokens_mask=True,
        )

        for i in range(len(tokenized['input_ids'])):
            for k in tokenized:
                flattened[k].append(tokenized[k][i])
            flattened['label'].append(label)

    return dict(flattened)

processed_dataset = dataset.map(
    handle_sample,
    remove_columns=dataset['test'].column_names,
    batch_size=64,
    batched=True,
    num_proc=8,
)


In [ ]:
config = RobertaConfig(
    vocab_size=tokenizer.vocab_size, 
    max_position_embeddings=MAX_LENGTH + 2, 
    num_attention_heads=4,
    num_hidden_layers=4,
    type_vocab_size=1,
    hidden_size=128,
    intermediate_size=2048,
)

model = RobertaForMaskedLM(config=config)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

train_ds = processed_dataset['train'].remove_columns('label')
test_ds = processed_dataset['test'].remove_columns('label')

train_args = TrainingArguments(
    output_dir="./MalBERTa",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64, 
    save_steps=10_000, 
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=train_args, 
    processing_class=tokenizer,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)

# trainer.train()

In [ ]:
import torch
import pandas as pd

def predict(token_ids):
    X = data_collator(torch.tensor(token_ids['input_ids']))
    preds = trainer.predict(X['input_ids'])
    
    Y_hat = tokenizer.batch_decode(preds.predictions.argmax(-1))
    Y = tokenizer.batch_decode(token_ids['input_ids'])

    df = pd.DataFrame(data={
        "Input": tokenizer.batch_decode(X['input_ids']),
        "Predicted": Y_hat,
        "Actual": Y,
    })

    return df
    
data = test_ds.select(range(10))
predict(data)

In [5]:
from transformers import RobertaForSequenceClassification
import torch

model = RobertaForSequenceClassification.from_pretrained("./MalBERTa")
data = processed_dataset['train'][0]

input_ids = torch.tensor(data['input_ids']).unsqueeze(0)
attention_mask = torch.tensor(data['attention_mask']).unsqueeze(0)
label = torch.tensor(data['label'])

model(input_ids, attention_mask, labels=label)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./MalBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=tensor(0.6620, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0293,  0.0339]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions, average="weighted", zero_division=0),
        "recall": recall_score(labels, predictions, average="weighted", zero_division=0),
        "f1": f1_score(labels, predictions, average="weighted", zero_division=0),
    }

train_args = TrainingArguments(
    output_dir="./MalBERTa-classifier",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=256, 
    per_device_eval_batch_size=256, 
    save_steps=10_000, 
    save_total_limit=2,
    eval_steps=100,
    eval_strategy="steps"
)

trainer = Trainer(
    model=model,
    args=train_args, 
    processing_class=tokenizer,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.evaluate()

NameError: name 'TrainingArguments' is not defined

In [ ]:
model.classifier(processed_dataset['test'])